In [6]:
import sqlite3
import pandas as pd
import ollama
import chromadb
import re
import unicodedata
from chromadb.utils import embedding_functions
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama as VannaOllama

# ==============================================================================
# FUNÇÃO AUXILIAR: NORMALIZAÇÃO
# ==============================================================================
def normalizar_texto(texto):
    """Remove acentos e deixa em minúsculo para buscas mais precisas (opcional, dependendo do BD)."""
    if not texto: return ""
    return "".join(c for c in unicodedata.normalize('NFD', texto)
                  if unicodedata.category(c) != 'Mn').lower()

# ==============================================================================
# 1. ANALISTA SQL (Vanna)
# ==============================================================================
class SQLAnalyst(ChromaDB_VectorStore, VannaOllama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        VannaOllama.__init__(self, config=config)

    def preparar_agente(self, db_path):
        self.connect_to_sqlite(db_path)
        
        if self.get_training_data().empty:
            print("   [SQL] Treinando inteligência de busca flexível...")
            
            # DDL Atualizado para refletir TODAS as colunas do seu DataFrame real
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER,
                banheiros INTEGER,
                garagem INTEGER,
                area DECIMAL,
                cidade VARCHAR(100),
                bairro VARCHAR(100),
                rua VARCHAR(200),
                numero VARCHAR(20),
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL,
                preco_condominio DECIMAL,
                aceita_pets INTEGER,
                imagem TEXT,
                codigo_bairro INTEGER,
                especificacao VARCHAR(100)
            );
            """)

            # Exemplos treinando a IA a ser flexível e usar LOWER/LIKE nativamente
            exemplos = [
                ("Tem algum loft em São Mateus?", 
                 "SELECT * FROM core_imovel WHERE (LOWER(especificacao) LIKE '%loft%' OR LOWER(titulo) LIKE '%loft%') AND LOWER(bairro) LIKE '%são mateus%'"),
                ("Quais casas tem no centro?", 
                 "SELECT * FROM core_imovel WHERE LOWER(especificacao) LIKE '%casa%' AND LOWER(bairro) LIKE '%centro%'"),
                ("Quero algo barato no Alto dos Passos", 
                 "SELECT * FROM core_imovel WHERE LOWER(bairro) LIKE '%alto dos passos%' ORDER BY preco_aluguel ASC LIMIT 3"),
                ("Apartamento com 3 quartos e 2 vagas de garagem",
                 "SELECT * FROM core_imovel WHERE quartos >= 3 AND garagem >= 2 AND LOWER(especificacao) LIKE '%apartamento%'")
            ]
            for q, s in exemplos:
                self.train(question=q, sql=s)

    def executar_consulta(self, pergunta):
        try:
            # Instrução adicional para reforçar consultas seguras (ILIKE/LOWER)
            pergunta_enriquecida = f"{pergunta} (Use LIKE e LOWER() para textos ao invés de '=')"
            sql = self.generate_sql(pergunta_enriquecida)
            
            print(f"   [DEBUG SQL] Gerado: {sql}")
            df = self.run_sql(sql)
            return df
        except Exception as e:
            print(f"   [ERRO SQL] Não foi possível executar a busca: {e}")
            return pd.DataFrame()

# ==============================================================================
# 2. ESPECIALISTA EM REGRAS (RAG)
# ==============================================================================
class RulesExpert:
    def __init__(self, chroma_client):
        self.embed_fn = embedding_functions.OllamaEmbeddingFunction(
            url="http://localhost:11434/api/embeddings", model_name="nomic-embed-text"
        )
        self.collection = chroma_client.get_or_create_collection(
            name="regras_v15", embedding_function=self.embed_fn
        )
        self._popular_regras()

    def _popular_regras(self):
        if self.collection.count() > 0: return
        print("   [RAG] Populando regras de negócio...")
        regras = {
            "docs": "Documentos: RG, CPF, Comprovante de Renda (3x o valor do aluguel) e Comprovante de Residência.",
            "garantia": "Garantias: Aceitamos Fiador, Seguro Fiança (Porto Seguro/CredPago) ou Título de Capitalização.",
            "horario": "Visitas: Segunda a sexta das 09h às 17h, e sábados das 09h às 12h.",
            "pets": "Pets: Depende da autorização do proprietário de cada imóvel, mas a maioria aceita pets de pequeno porte."
        }
        for k, v in regras.items():
            self.collection.add(ids=[k], documents=[v])

    def buscar_regra(self, pergunta):
        res = self.collection.query(query_texts=[pergunta], n_results=1)
        return res['documents'][0][0] if res['documents'] else ""

# ==============================================================================
# 3. BIA PERSONA (A Secretária Proativa)
# ==============================================================================
class BiaPersona:
    def __init__(self):
        self.model = 'llama3.1:8b'

    def responder(self, pergunta, df=None, regra="", historico="", lead=""):
        imoveis_str = "Nenhum imóvel encontrado no momento para este critério específico."
        
        # Iteração segura do DataFrame com tratamento para colunas ausentes
        if df is not None and not df.empty:
            imoveis_str = ""
            for _, r in df.iterrows():
                bairro = r.get('bairro', 'Bairro não informado')
                preco = r.get('preco_aluguel', 'Preço sob consulta')
                quartos = r.get('quartos', '?')
                titulo = r.get('titulo', 'Imóvel')
                imoveis_str += f"- {titulo} ({bairro}) | {quartos} quarto(s) | R$ {preco}\n"

        system_prompt = f"""
        Você é a Bia, secretária da Imobiliária Juiz de Fora. 
        Seu tom: Profissional, prestativo e cordial.

        DADOS DO BANCO:
        {imoveis_str}

        CONHECIMENTO ADICIONAL (REGRAS):
        {regra}

        {f'ALERTA: O cliente deixou um telefone: {lead}' if lead else ''}

        MISSÃO:
        1. Se o banco retornou imóveis, apresente-os de forma agradável e convide para uma visita.
        2. Se o banco NÃO retornou nada, seja proativa: sugira olhar bairros vizinhos ou peça para anotar o contato.
        3. Se a pergunta for sobre regras (documentos, fiador), use o CONHECIMENTO ADICIONAL para responder.
        4. Nunca diga 'Não temos' de forma seca.
        5. Seja concisa e use bullet points.
        """
        
        prompt = f"{system_prompt}\n\nHistórico Recente:\n{historico}\n\nUsuário: {pergunta}\nBia:"
        try:
            resposta = ollama.generate(model=self.model, prompt=prompt, options={'temperature': 0.3})
            return resposta['response']
        except Exception as e:
            return "Sinto muito, tive um probleminha técnico de conexão. Pode repetir a pergunta?"

# ==============================================================================
# 4. ORQUESTRADOR
# ==============================================================================
class Orchestrator:
    def __init__(self, db_path):
        self.chroma = chromadb.PersistentClient(path="./chroma_db")
        # Usando um LLM forte para código no Vanna
        self.sql_agent = SQLAnalyst(config={"model": "qwen2.5-coder:7b", "path": "./vanna_db"})
        self.sql_agent.preparar_agente(db_path)
        self.rules_agent = RulesExpert(self.chroma)
        self.bia = BiaPersona()
        self.historico = []

    def processar(self, entrada):
        # Prompt mais restrito para o Qwen não inventar texto na classificação
        intencao_prompt = f"""
        Classifique a intenção do usuário em apenas UMA das seguintes categorias:
        - BUSCAR (se ele quer procurar imóveis, ver preços, bairros, etc.)
        - REGRAS (se ele pergunta sobre fiador, documentos, pets, horários)
        - CONVERSAR (se é apenas um oi, saudação ou conversa fiada)
        
        Responda APENAS com a palavra da categoria. Nenhuma palavra a mais.
        Usuário: '{entrada}'
        """
        classe_response = ollama.generate(model="qwen2.5-coder:7b", prompt=intencao_prompt, options={'temperature': 0.0})
        classe = classe_response['response'].strip().upper()

        print(f"   [DEBUG INTENÇÃO] {classe}")

        df, regra, lead = None, "", ""
        
        # Detecção básica de lead (telefone)
        if re.search(r'\d{8,}', entrada):
            lead = "Interessado deixou contato telefônico."

        if "BUSCAR" in classe:
            df = self.sql_agent.executar_consulta(entrada)
        elif "REGRAS" in classe:
            regra = self.rules_agent.buscar_regra(entrada)

        # Monta o histórico das últimas 2 interações
        hist_str = "\n".join([f"{h['role']}: {h['content']}" for h in self.historico[-2:]])
        
        resposta = self.bia.responder(entrada, df, regra, hist_str, lead)

In [7]:
import sqlite3
import pandas as pd

# Conecta ao banco de dados
conn = sqlite3.connect('db.sqlite3')

# Carrega os dados da tabela core_imovel para um DataFrame
df = pd.read_sql_query("SELECT * FROM core_imovel", conn)

# Fecha a conexão
conn.close()

# Exibe as primeiras linhas
print(df)

      id                                 titulo  \
0     59  Apartamento Confortável em São Mateus   
1     60            Cobertura Duplex São Mateus   
2     62       Casa Charmosa no Alto dos Passos   
3     63       Apartamento Luxo Alto dos Passos   
4     64                        Loft Industrial   
..   ...                                    ...   
105  170            Studio Reformado no Benfica   
106  171      Studio Aconchegante no Cascatinha   
107  172    Cobertura Moderno no Manoel Honório   
108  173        Cobertura Reformado no Granbery   
109  174               Casa Espaçoso no Benfica   

                                             descricao  quartos  banheiros  \
0    Ótimo apartamento de frente, sol da manhã, pró...        2          2   
1    Cobertura incrível com vista panorâmica, área ...        3          3   
2    Casa antiga reformada, com quintal arborizado ...        3          2   
3    Apartamento de alto padrão, um por andar, acab...        4          4 

In [8]:
df

,id,titulo,descricao,quartos,banheiros,garagem,area,cidade,bairro,rua,numero,preco_aluguel,preco_iptu,preco_condominio,aceita_pets,imagem,codigo_bairro,especificacao
0,59,Apartamento Confortável em São Mateus,"Ótimo apartamento de frente, sol da manhã, pró...",2,2,1,85.00,Juiz de Fora,São Mateus,Rua Padre Café,120,1800.00,150.00,350.00,1,None,200,apartamento
1,60,Cobertura Duplex São Mateus,"Cobertura incrível com vista panorâmica, área ...",3,3,2,160.00,Juiz de Fora,São Mateus,Rua Dr. Romualdo,450,3200.00,300.00,600.00,1,None,200,apartamento
2,62,Casa Charmosa no Alto dos Passos,"Casa antiga reformada, com quintal arborizado ...",3,2,2,200.00,Juiz de Fora,Alto dos Passos,Rua Severiano Sarmento,88,2500.00,220.00,0.00,1,None,220,casa
3,63,Apartamento Luxo Alto dos Passos,"Apartamento de alto padrão, um por andar, acab...",4,4,3,220.00,Juiz de Fora,Alto dos Passos,Rua Dom Viçoso,300,4500.00,500.00,1200.00,1,None,220,apartamento
4,64,Loft Industrial,"Loft com pegada industrial, pé direito duplo e...",1,1,1,70.00,Juiz de Fora,Alto dos Passos,Rua Morais e Castro,55,1900.00,120.00,400.00,0,None,220,apartamento
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,170,Studio Reformado no Benfica,Excelente studio situado na Rua Martins Barbos...,4,1,0,112.50,Juiz de Fora,Benfica,Rua Martins Barbosa,2492,3736.88,373.69,473.05,0,None,NaN,studio
106,171,Studio Aconchegante no Cascatinha,Excelente studio situado na Rua Tom Fagundes. ...,2,1,1,216.42,Juiz de Fora,Cascatinha,Rua Tom Fagundes,1440,1850.97,185.10,322.57,0,None,NaN,studio
107,172,Cobertura Moderno no Manoel Honório,Excelente cobertura situado na Av. Governador ...,2,3,0,223.90,Juiz de Fora,Manoel Honório,Av. Governador Valadares,243,1982.07,198.21,433.79,1,None,NaN,cobertura
108,173,Cobertura Reformado no Granbery,Excelente cobertura situado na Rua Barão de Sa...,4,1,2,214.36,Juiz de Fora,Granbery,Rua Barão de Santa Helena,1993,4159.83,415.98,991.02,0,None,NaN,cobertura


In [9]:
df[df['especificacao']=='loft']

,id,titulo,descricao,quartos,banheiros,garagem,area,cidade,bairro,rua,numero,preco_aluguel,preco_iptu,preco_condominio,aceita_pets,imagem,codigo_bairro,especificacao
14,79,Loft Próximo ao comércio no São Pedro,Excelente loft situado na Av. Presidente Costa...,1,3,1,248.12,Juiz de Fora,São Pedro,Av. Presidente Costa e Silva,766,4213.65,421.37,1080.34,0,None,NaN,loft
15,80,Loft Reformado no Centro,Excelente loft situado na Rua Marechal Deodoro...,2,3,1,230.91,Juiz de Fora,Centro,Rua Marechal Deodoro,957,1288.19,128.82,1077.59,0,None,NaN,loft
17,82,Loft Espaçoso no São Pedro,Excelente loft situado na Av. Presidente Costa...,3,1,1,214.52,Juiz de Fora,São Pedro,Av. Presidente Costa e Silva,1736,933.84,93.38,624.30,0,None,NaN,loft
21,86,Loft Reformado no Alto dos Passos,Excelente loft situado na Rua Morais e Castro....,1,3,0,90.64,Juiz de Fora,Alto dos Passos,Rua Morais e Castro,233,4327.44,432.74,1091.61,0,None,NaN,loft
23,88,Loft Moderno no Bom Pastor,Excelente loft situado na Rua Senador Salgado ...,3,1,1,155.24,Juiz de Fora,Bom Pastor,Rua Senador Salgado Filho,1231,1142.75,114.28,374.99,0,None,NaN,loft
27,92,Loft Luxuoso no Cascatinha,Excelente loft situado na Rua Dr. Paulo Japias...,4,2,1,244.16,Juiz de Fora,Cascatinha,Rua Dr. Paulo Japiassu Coelho,1332,1704.00,170.40,364.57,1,None,NaN,loft
32,97,Loft Luxuoso no Manoel Honório,Excelente loft situado na Av. Governador Valad...,1,2,1,67.29,Juiz de Fora,Manoel Honório,Av. Governador Valadares,251,3398.90,339.89,276.99,0,None,NaN,loft
58,123,Loft Moderno no São Mateus,Excelente loft situado na Rua São Mateus. Mode...,2,2,0,111.76,Juiz de Fora,São Mateus,Rua São Mateus,1373,3300.92,330.09,690.27,1,None,NaN,loft
61,126,Loft Reformado no Granbery,Excelente loft situado na Rua Barão de Santa H...,1,1,2,199.02,Juiz de Fora,Granbery,Rua Barão de Santa Helena,445,3213.19,321.32,172.32,0,None,NaN,loft
73,138,Loft Reformado no São Mateus,Excelente loft situado na Rua Dr. Romualdo. Re...,4,2,1,165.62,Juiz de Fora,São Mateus,Rua Dr. Romualdo,676,3800.75,380.08,285.18,1,None,NaN,loft
